
# Probabilistic Forecasting with `sktime`: Distribution Forecast

## Notebook Goal: 
* Provide an example workflow of a distribution forecast with BGL data using sktime.
* Demonstrate how to use Optuna for a single forecasting type.

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data Load

In [10]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "..", "..")))


In [13]:
from src.data.data_loader import load_data
import pandas as pd

# Load the data with specified columns
df = load_data(data_source_name="kaggle_brisT1D", dataset_type="train")

/Users/tonychan/GlucoseML/nocturnal-hypo-gly-prob-forecast/src/data/data_loader.py:32: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.


In [11]:
# Specify the columns to keep
id_vars = ["time"]
feature_cols = ["bg", "insulin", "carbs", "hr", "steps", "cals", "activity"]


features = {}
for feature in feature_cols:
    features[feature] = [col for col in df.columns if feature in col]


def process_row(row, features):
    first_feature = True
    row_df = None
    row_df_temp = pd.DataFrame(row).T

    for feature, feature_cols in features.items():
        melted = row_df_temp.melt(
            id_vars=id_vars,
            value_vars=feature_cols,
            var_name="time_offset",
            value_name=feature,
        )

        if first_feature:
            row_df = melted
            first_feature = False
            melted["time_offset"] = melted["time_offset"].str.replace(f"{feature}", "")
        else:
            row_df[feature] = melted[feature]

    return row_df


def process_patient(patient_df: pd.DataFrame):
    # For each patient, make each row in a dataframe
    rows = []
    print("There are ", len(patient_df), " rows in this patient")
    for id, row in patient_df.iterrows():
        print("Processing row ", id)
        rows.append(process_row(row, features))

        if id == 200:
            break

    return rows


def process_patients(df: pd.DataFrame):
    patients = {}
    # Process each patient
    for p_num, patient_df in df.groupby("p_num"):
        # Key is the patient id
        patient_df.drop(columns=["id", "p_num"], inplace=True)
        patients[p_num] = process_patient(patient_df)

        # Only process the first patient
        break

    return patients


# TODO:
# 1. Remove if (id == 200):
# 2. Remove the break
# 3. Find a way to improve this
# 4. process_patients should take patient number as an argument
# 5. We can retrieve + (which is the predicted value) in other functions
processed_patients = process_patients(df)
processed_patients["p01"][0]

There are  8459  rows in this patient
Processing row  0
Processing row  1
Processing row  2
Processing row  3
Processing row  4
Processing row  5
Processing row  6
Processing row  7
Processing row  8
Processing row  9
Processing row  10
Processing row  11
Processing row  12
Processing row  13
Processing row  14
Processing row  15
Processing row  16
Processing row  17
Processing row  18
Processing row  19
Processing row  20
Processing row  21
Processing row  22
Processing row  23
Processing row  24
Processing row  25
Processing row  26
Processing row  27
Processing row  28
Processing row  29
Processing row  30
Processing row  31
Processing row  32
Processing row  33
Processing row  34
Processing row  35
Processing row  36
Processing row  37
Processing row  38
Processing row  39
Processing row  40
Processing row  41
Processing row  42
Processing row  43
Processing row  44
Processing row  45
Processing row  46
Processing row  47
Processing row  48
Processing row  49
Processing row  50
Pro

,time,time_offset,bg,insulin,carbs,hr,steps,cals,activity
0,06:10:00,-5:55,NaN,0.0083,NaN,NaN,NaN,NaN,NaN
1,06:10:00,-5:50,NaN,0.0083,NaN,NaN,NaN,NaN,NaN
2,06:10:00,-5:45,9.6,0.0083,NaN,NaN,NaN,NaN,NaN
3,06:10:00,-5:40,NaN,0.0083,NaN,NaN,NaN,NaN,NaN
4,06:10:00,-5:35,NaN,0.0083,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
68,06:10:00,-0:15,16.2,0.0583,NaN,NaN,NaN,NaN,NaN
69,06:10:00,-0:10,NaN,0.0583,NaN,NaN,NaN,NaN,NaN
70,06:10:00,-0:05,NaN,0.0417,NaN,NaN,NaN,NaN,NaN
71,06:10:00,-0:00,15.1,0.0417,NaN,NaN,NaN,NaN,NaN


In [5]:
from sktime.forecasting.arima import ARIMA

# step 1: data specification
y = df["bg-0:00"]
# step 2: specifying forecasting horizon
fh = [1, 2, 3]
# step 3: specifying the forecasting algorithm
forecaster = ARIMA()
# step 4: fitting the forecaster
forecaster.fit(y, fh=[1, 2, 3])
# step 5: querying predictions
y_pred = forecaster.predict()

# for probabilistic forecasting:
#   call a probabilistic forecasting method after or instead of step 5
y_pred_int = forecaster.predict_interval(coverage=0.95)
y_pred_int

: 

In [7]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)